## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

## Import Data

In [3]:
titles_imdb = pd.read_csv('../data/title.basics.tsv.gz', sep='\t', compression='gzip')
ratings_imdb = pd.read_csv('../data/title.ratings.tsv.gz', sep='\t', compression='gzip')
budgets_kaggle = pd.read_csv('../data/kaggle_movies_metadata.csv')

In [4]:
titles_imdb.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [5]:
ratings_imdb.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
2,tt0000003,6.5,1734
3,tt0000004,5.6,174
4,tt0000005,6.2,2545


In [6]:
budgets_kaggle.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
# Importing CMU Datasets
movie = pd.read_csv('./../data/MovieSummaries/movie.metadata.tsv', 
                 sep='\t',
                 names=['Wikipedia_movie_ID',
                        'Freebase_movie_ID',
                        'Movie_name',
                        'Movie_release_date',
                        'Movie_box_office_revenue',
                        'Movie_runtime',
                        'Movie_languages',
                        'Movie_countries',
                        'Movie_genres'],
                 header=None)

character = pd.read_csv('./../data/MovieSummaries/character.metadata.tsv', 
                 sep='\t',
                 names=['Wikipedia_movie_ID',
                        'Freebase_movie_ID',
                        'Movie_release_date',
                        'Character_name',
                        'Actor_date_of_birth',
                        'Actor_gender',
                        'Actor_height',
                        'Actor_ethnicity',
                        'Actor_name',
                        'Actor_age_at_movie_release',
                        'Freebase_character/actor_map_ID',
                        'Freebase_character_ID',
                        'Freebase_actor_ID'],
                 header=None)

In [8]:
movie.head()

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"


In [9]:
character.head()

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_release_date,Character_name,Actor_date_of_birth,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age_at_movie_release,Freebase_character/actor_map_ID,Freebase_character_ID,Freebase_actor_ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg


# Data cleaning

### Date change

In [10]:
# Movie release dates and actor birth dates to datetime format and keep only release year
character.Movie_release_date = pd.to_datetime(character['Movie_release_date'],
                                              errors='coerce').dt.year

movie.Movie_release_date = pd.to_datetime(movie['Movie_release_date'],
                                              errors='coerce').dt.year

In [11]:
# Sort dataframes by ascending release year
character.sort_values(by=['Movie_release_date'], ascending=True, inplace=True)

movie.sort_values(by=['Movie_release_date'], ascending=True, inplace=True)

### Null value removal

In [12]:
# NaN percentage for character dataset for each column
character.isnull().sum() * 100 / character.shape[0]

Wikipedia_movie_ID                  0.000000
Freebase_movie_ID                   0.000000
Movie_release_date                  2.221142
Character_name                     57.220488
Actor_date_of_birth                23.552763
Actor_gender                       10.120288
Actor_height                       65.645740
Actor_ethnicity                    76.466542
Actor_name                          0.272484
Actor_age_at_movie_release         35.084064
Freebase_character/actor_map_ID     0.000000
Freebase_character_ID              57.218269
Freebase_actor_ID                   0.180842
dtype: float64

In [13]:
# Remove unnecessary columns from character dataset
character = character[['Wikipedia_movie_ID', 'Actor_name', 'Actor_gender', 'Actor_date_of_birth']]

In [14]:
# Dropping all rows from the character dataset that have NaN values
char_no_nan = character.dropna(axis=0, how='any')

In [15]:
# Nan percentage for movie dataset for each column
movie.isnull().sum() * 100 / movie.shape[0]

Wikipedia_movie_ID           0.000000
Freebase_movie_ID            0.000000
Movie_name                   0.000000
Movie_release_date           8.444966
Movie_box_office_revenue    89.722416
Movie_runtime               25.018045
Movie_languages              0.000000
Movie_countries              0.000000
Movie_genres                 0.000000
dtype: float64

In [16]:
# Remove unnecessary columns from movie dataset
movie = movie[['Wikipedia_movie_ID', 
               'Movie_name', 
               'Movie_release_date', 
               'Movie_box_office_revenue',
               'Movie_runtime', 
               'Movie_countries', 
               'Movie_genres']]

In [17]:
# Dropping all rows from the movie dataset that have NaN values
movie_no_nan = movie.dropna(0, inplace= False, how='any')

In [18]:
# Extracting budget data from the Kaggle dataset
budgets_kaggle = budgets_kaggle[['original_title', 'budget']]

In [19]:
# Converting movies with budgets of zero to NaN and printing percentage of NaN data
budgets_kaggle['budget'] = budgets_kaggle['budget'].replace('0', np.nan)
budgets_kaggle.isna().sum() * 100 / budgets_kaggle.shape[0]

original_title     0.000000
budget            80.440329
dtype: float64

In [20]:
# Dropping all rows from the budget dataset that have NaN values
budget_no_nan = budgets_kaggle.dropna()

# Merging Datasets

In [21]:
# Merging movie titles dataset with ratings dataset
first_merge = titles_imdb.merge(ratings_imdb, how='inner')

In [22]:
# Droping unnecessary columns
imdb = first_merge[['primaryTitle', 'averageRating', 'numVotes']]

In [25]:
# Removing duplicate film titles and averaging the duplicate ratings
mean_rating_imdb = first_merge[['primaryTitle', 'averageRating']].groupby('primaryTitle').mean()

In [26]:
# Removing duplicate film titles and adding together the duplicate votes
total_vote_imdb = first_merge[['primaryTitle', 'numVotes']].groupby('primaryTitle').sum()

In [27]:
# Merging the two above dataframes
no_dupe_imdb = mean_rating_imdb.merge(total_vote_imdb, 'inner', 'primaryTitle')

In [28]:
# Merging the rating/votes dataframe with movie metadata with no nan values
rated_data = movie_no_nan.merge(no_dupe_imdb, 'inner', left_on='Movie_name', right_on='primaryTitle')

## Parsing the genres and movie countries

In [29]:
# Parsing the dictionaries provided for the columns Movie_genres and Movie_countries
rated_data['genres'] = rated_data['Movie_genres'].apply(lambda x : re.split(r':|,', x)[1::2])
rated_data['countries'] = rated_data['Movie_countries'].apply(lambda x : re.split(r':|,', x)[1::2])
rated_dat = rated_data.drop(['Movie_countries', 'Movie_genres'], axis = 1) #run once

In [30]:
# Renaming column to avoid merge issues
budget_no_nan = budget_no_nan.rename(columns={'original_title': 'Movie_name'})

In [31]:
# Merging dataset with budget information and the ratings dataset
budget = pd.merge(budget_no_nan, rated_dat)

In [32]:
# Converting the budget column from string to numerical value
budget['budget'] = pd.to_numeric(budget['budget'])

In [33]:
# Creating a Profit column
budget['profit'] = budget['Movie_box_office_revenue'] - budget['budget']

In [34]:
# Merging the profit dataset with character dataset
profit = pd.merge(char_no_nan, budget)
profit = profit.drop(['Wikipedia_movie_ID'], axis=1)
profit.head()

,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,budget,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries,profit
0,Elmer Clifton,M,1890-03-14,The Birth of a Nation,100000,1915.0,50000000.0,190.0,6.6,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",49900000.0
1,Elmer Clifton,M,1890-03-14,The Birth of a Nation,8500000,1915.0,50000000.0,190.0,6.6,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",41500000.0
2,Robert Harron,M,1893-04-12,The Birth of a Nation,100000,1915.0,50000000.0,190.0,6.6,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",49900000.0
3,Robert Harron,M,1893-04-12,The Birth of a Nation,8500000,1915.0,50000000.0,190.0,6.6,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",41500000.0
4,Walter Long,M,1879-03-05,The Birth of a Nation,100000,1915.0,50000000.0,190.0,6.6,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",49900000.0


In [35]:
# The final merge with character/actor data and the movie dataset
data = pd.merge(char_no_nan, rated_dat)
data = data.drop(['Wikipedia_movie_ID'], axis=1)
data.head()

,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries
0,Dustin Farnum,M,1874-05-27,The Squaw Man,1914.0,244700.0,74.0,5.833333,1418,"[ ""Silent film"", ""Western"", ""Drama"", ""Indie...","[ ""United States of America""}]"
1,Elmer Clifton,M,1890-03-14,The Birth of a Nation,1915.0,50000000.0,190.0,6.600000,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]"
2,Robert Harron,M,1893-04-12,The Birth of a Nation,1915.0,50000000.0,190.0,6.600000,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]"
3,Walter Long,M,1879-03-05,The Birth of a Nation,1915.0,50000000.0,190.0,6.600000,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]"
4,Violet Wilkey,F,1903-01-10,The Birth of a Nation,1915.0,50000000.0,190.0,6.600000,46186,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]"


### Adjusting for inflation

In [36]:
cf = pd.read_csv('../data/inflation_correction.csv') #inflation correction

inflation_match_profit = profit.merge(cf, how = 'inner', left_on='Movie_release_date', right_on='Year')
profit['profit_adj'] = profit['profit']/inflation_match_profit['CF']
inflation_match_data = data.merge(cf, how = 'inner', left_on='Movie_release_date', right_on='Year')
data['box_office_adj'] = data['Movie_box_office_revenue']/inflation_match_data['CF']

display(profit.sample(), data.sample())

,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,budget,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries,profit,profit_adj
2722,Bette Davis,F,1908-04-05,What Ever Happened to Baby Jane?,980000,1962.0,9000000.0,135.0,6.966667,58101,"[ ""Thriller"", ""Drama"", ""Horror"", ""Psycholog...","[ ""United States of America""}]",8020000.0,6.169231e+07


,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries,box_office_adj
3104,Ann Miller,F,1923-04-12,Two Tickets to Broadway,1951.0,2000000.0,106.0,5.5,564,"[ ""Musical""}]","[ ""United States of America""}]",1.818182e+07


### Reading cleaned dataset to CSV

In [37]:
profit.to_csv('../data/profit_data.csv') #useful for movie regression
data.to_csv('../data/data.csv')